<img src="logo.png">

# Regularización

Recordemos que una regresión lineal es un modelo de la forma $$\widehat{y}=\alpha+\beta_1x_1+\beta_2x_2+...+\beta_nx_n+\epsilon$$ donde nuestro objetivo es hallar $\alpha$ y $\beta=(\beta_1,\beta_2,...,\beta_n)$ que minimicen la expresión $$\sum_{i=1}^n(y_i-\alpha-\beta_ix_i)^2.$$ 

En el caso de regresión polinomial tenemos:

<img src="ml79.png">

## Sobreajuste y generalización

<img src="ml80.png">

## Regularización en modelos lineales

<img src="ml81.png">

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_validate

In [ ]:
vehiculos = pd.read_csv("vehiculos_procesado.csv")
datos_entrenamiento = vehiculos[["desplazamiento","cilindros","consumo"]]

objetivo = vehiculos["co2"]

In [ ]:
datos_entrenamiento.shape

In [ ]:
datos_entrenamiento.head()

In [ ]:
from sklearn.linear_model import (LinearRegression, Lasso,
                                  Ridge, ElasticNet)


In [ ]:
modelo_ols = LinearRegression()
modelo_ols.fit(datos_entrenamiento, objetivo)

modelo_ols.coef_

Para medir la complejidad del modelo, vamos a usar la funcion de [`numpy.linalg.norm`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.norm.html) que es una función que calcula varios tipos de normas.

In [ ]:
def norma_l1(coeficientes):
    return np.linalg.norm(coeficientes, ord=1)

def norma_l2(coeficientes):
    return np.linalg.norm(coeficientes, ord=2)

print(norma_l1(modelo_ols.coef_))
print(norma_l2(modelo_ols.coef_))

In [ ]:
def norma_l1_cv(estimator, X, y):
    return norma_l1(estimator.coef_)

def norma_l2_cv(estimator, X, y):
    return norma_l2(estimator.coef_)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
PolynomialFeatures?

In [ ]:
transformador_polinomial = PolynomialFeatures(5)

In [ ]:
transformador_polinomial.fit(datos_entrenamiento)

In [ ]:
transformador_polinomial?

In [ ]:
variables_polinomiales = transformador_polinomial.transform(
    datos_entrenamiento)

In [ ]:
variables_polinomiales?

In [ ]:
variables_polinomiales.shape

In [ ]:
datos_entrenamiento.loc[0]

In [ ]:
variables_polinomiales[0]

In [ ]:
variables_polinomiales = PolynomialFeatures(5).fit_transform(
    datos_entrenamiento)

In [ ]:
variables_polinomiales.shape

Ahora vamos a evaluar los distintos tipos de regularizacion

**Modelo OLS con variables polinomiales**

In [ ]:
RESULTADOS = {}

In [ ]:
modelo_ols = LinearRegression()
modelo_ols.fit(variables_polinomiales, objetivo)
print(modelo_ols.coef_)

RESULTADOS["ols"] = {
    "norma_l1": norma_l1(modelo_ols.coef_),
    "norma_l2": norma_l2(modelo_ols.coef_),
}

In [ ]:
RESULTADOS

**Modelo Regularización L1 con variables polinomiales**

In [ ]:
Lasso?

In [ ]:
modelo_l1 = Lasso(alpha=1.0, tol=0.01, max_iter=5000)
modelo_l1.fit(variables_polinomiales, objetivo)
print(modelo_l1.coef_)

RESULTADOS["regularizacion_l1"] = {
    "norma_l1": norma_l1(modelo_l1.coef_),
    "norma_l2": norma_l2(modelo_l1.coef_),
}

In [ ]:
RESULTADOS["regularizacion_l1"]

**Modelo Regularización L2 (Ridge)  con variables polinomiales**

In [ ]:
modelo_l2 = Ridge(alpha=1.0, tol=0.01, max_iter=5000)
modelo_l2.fit(variables_polinomiales, objetivo)

print(modelo_l2.coef_)
RESULTADOS["regularizacion_l2"] = {
    "norma_l1": norma_l1(modelo_l2.coef_),
    "norma_l2": norma_l2(modelo_l2.coef_),
}

In [ ]:
RESULTADOS["regularizacion_l2"]

**Regularización Elasticnet con variables polinomiales**

In [ ]:
ElasticNet?

In [ ]:
modelo_elasticnet = ElasticNet(l1_ratio=0.5, tol=0.01,max_iter=5000)
modelo_elasticnet.fit(variables_polinomiales, objetivo)
print(modelo_elasticnet.coef_)

RESULTADOS["regularizacion_elasticnet"] = {
    "norma_l1": norma_l1(modelo_elasticnet.coef_),
    "norma_l2": norma_l2(modelo_elasticnet.coef_),
}

In [ ]:
RESULTADOS["regularizacion_elasticnet"]

In [ ]:
pd.set_option("display.float_format", lambda x: str(round(x,6)))

In [ ]:
resultados_df = pd.DataFrame(RESULTADOS).T
l1_ols = resultados_df.loc["ols", "norma_l1"]
l2_ols = resultados_df.loc["ols", "norma_l2"]

resultados_df["pct_reduccion_l1"] = 1-resultados_df.norma_l1 / l1_ols
resultados_df["pct_reduccion_l2"] = 1-resultados_df.norma_l2 / l2_ols

resultados_df